In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, AutoPeftModelForCausalLM

/storage/ice1/2/3/ckniffin6/conda_envs/hf/lib/python3.12/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
from huggingface_hub import login
login()

In [3]:
model = AutoPeftModelForCausalLM.from_pretrained("/home/hice1/ckniffin6/scratch/DL/project/adapters/Qlora_rank8").to("cuda")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
# Load the saved model
modelName = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(modelName)

base_prompt_template: str = """
    You are a financial analyst designed to answer questions about business and finance. Your job is to reply to questions about finance topics and provide advice.

    Instruction: {query}
    
    Output:
"""
    

In [5]:
queries = ["If I take out a $35000 loan with a 5% APR how many $500 payments would it take to pay it back?",
           "Assuming an inital investment of $100,000, an expected cash flow of $7000 per month, an interest rate of 7% and 3 years before the project is complete what is the net present value?",
           "You want to save up over 4 years for you child's college. If the cost of attendance is $100000 and you deposit money each month into a savings account with a 4% annual interest rate, how much do you have to deposit each month?",
           "How does a business compute net income?",
           "What is the rule of 72?",
           "What is CPI and how does it tell us what the inflation rate is?"]    

In [6]:
summary1 = '"As of March 31, 2024, the Company had aggregate outstanding notes of $14,016 million, net of $61 million of issuance costs, with varying maturities (the "Notes"). Of the outstanding balance, $799 million, net of issuance costs, is classified as short-term debt on the Consolidated Balance Sheets. As of December 31, 2023, the Company had aggregate outstanding notes of $14,543 million, net of $65 million of issuance costs. Each of the Notes were issued at par and are senior unsecured obligations of the Company. Interest is payable semi-annually at fixed rates. A portion of the outstanding Notes is denominated in foreign currency (comprised of €5,170 million) and is remeasured into U.S. dollars at each balance sheet date (with remeasurement gain totaling $131 million for the three months ended March 31, 2024)."  Based on the preceding text how much short term debt does the company have?'

In [7]:
summary2 = '"We anticipate our full-year 2024 capital expenditures will be in the range of $30-37 billion, a $2 billion increase of the high end of our prior range. We expect growth will be driven by investments in servers, including both AI and non-AI hardware, and data centers as we ramp up construction on sites with our previously announced new data center architecture. Our updated outlook reflects our evolving understanding of our artificial intelligence (AI) capacity demands as we anticipate what we may need for the next generations of foundational research and product development. While we are not providing guidance for years beyond 2024, we expect our ambitious long-term AI research and product development efforts will require growing infrastructure investments beyond this year." Based on this text what are some risks the that could cause its expenditures to be greater than the predicted $2 billion?'

In [8]:
summary3 = '"Product Liability We recorded liabilities of $596 million and $561 million in Accrued liabilities and Other liabilities at September 30, 2023 and December 31, 2022 for the expected cost of all known product liability claims, plus an estimate of the expected cost for product liability claims that have already been incurred and are expected to be filed in the future for which we are self-insured. It is reasonably possible that our accruals for product liability claims may increase in future periods in material amounts, although we cannot estimate a reasonable range of incremental loss based on currently available information. We believe that any judgment against us involving our products for actual damages will be adequately covered by our recorded accruals and, where applicable, excess liability insurance coverage." How much did the company accrue in total liabilities according to the preceeding text?'

In [9]:
query = base_prompt_template.format(query=queries[0])

modelInput = tokenizer(query, return_tensors="pt").to("cuda")

outputs = model.generate(**modelInput, max_new_tokens = 500, do_sample = True)
print(tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
    You are a financial analyst designed to answer questions about business and finance. Your job is to reply to questions about finance topics and provide advice.

    Instruction: If I take out a $35000 loan with a 5% APR how many $500 payments would it take to pay it back?
    
    Output:
<s> One thing a lot of people don't realize about loans is they are not paid off at a uniform rate. The very start of the payment is all interest, as you've no principle to pay off yet. The bulk of the payment goes against principle, while the very end is all the interest that's left. To calculate how many months until it's paid off, you need the payment, the remaining balance (50k), and the annual percentage rate. A simple google search for an amortization calculator will show the monthly breakdown, and at the very end it'll note total payments/months to payoff. You'll see a chart very like this: https://www.investopedia.com/tools/calculators/mortgage/mortgage-calculator.asp#tab=1 The $482.9

In [10]:
query = base_prompt_template.format(query=queries[1])

modelInput = tokenizer(query, return_tensors="pt").to("cuda")

outputs = model.generate(**modelInput, max_new_tokens = 500, do_sample = True)
print(tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
    You are a financial analyst designed to answer questions about business and finance. Your job is to reply to questions about finance topics and provide advice.

    Instruction: Assuming an inital investment of $100,000, an expected cash flow of $7000 per month, an interest rate of 7% and 3 years before the project is complete what is the net present value?
    
    Output:
<s> The equation to find the NPV is: 
NPV=Ca + Pmt(Rate,N,PV) + FV(1+rate)^N 
Where: Pmt = periodic payment, N = number of periods, rate= rate per payment period, FV = future value 
NPV = $100,000 + $7000*12*(7%) + $0 *(1+7%)^36 
NPV = $48,425.76 (rounded) 
Therefore, according to this equation, the NPV is a positive number indicating that this would be a good project. 
Note: The equation for NPV assumes the cash flows are constant and continuous, I have taken this into account above by multiplying total payments by 12 and then by 7% for a monthly rate and also included 36 payments to match the timeframe in

In [11]:
query = base_prompt_template.format(query=queries[2])

modelInput = tokenizer(query, return_tensors="pt").to("cuda")

outputs = model.generate(**modelInput, max_new_tokens = 500, do_sample = True)
print(tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
    You are a financial analyst designed to answer questions about business and finance. Your job is to reply to questions about finance topics and provide advice.

    Instruction: You want to save up over 4 years for you child's college. If the cost of attendance is $100000 and you deposit money each month into a savings account with a 4% annual interest rate, how much do you have to deposit each month?
    
    Output:
<s> £356.17 per month. Using the following equation: 1.04^48 - 1 = 0.4806139627731991 0.4806139627731991 = Interest 0.5206139627731991 = Payment P(0.04^4 -1) = Payment (x)=P(1-r^t) 0.5206139627731991 = £5,774.20  P(1 - 0.04^4) = £5,774.20 P = £5,774.20/114 months = £356.17 per month. (Source: equation for calculating payment over time) https://www.fool.co.uk/saving/savings-account/calculator/interest-calculator.asp?A=3.9175283699894115&N=14&p=1&i=0.04080638619476755%2C0.042279525654440815,0.04375276411316816,0.04522600265190549,&l=&t=&d=&cs=&e=&x=&a=&o=&p=1&v=3.9

In [12]:
query = base_prompt_template.format(query=queries[3])

modelInput = tokenizer(query, return_tensors="pt").to("cuda")

outputs = model.generate(**modelInput, max_new_tokens = 500, do_sample = True)
print(tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
    You are a financial analyst designed to answer questions about business and finance. Your job is to reply to questions about finance topics and provide advice.

    Instruction: How does a business compute net income?
    
    Output:
<s> Net income is calculated as the difference between a company's total revenues and total expenses over a specified period of time. To calculate net income, a company must account for all income and expenses, including costs of goods sold, operating expenses, interest and taxes. Once all revenues and expenses are accounted for, the company subtracts its total expenses from its total revenues to get its net income. This figure, also known as the bottom line, is used to measure a company's profitability. Companies that have positive net income are profitable and can continue to grow, while companies with negative net income are not profitable and may need to make adjustments to improve their financial standing. This is especially true if a compan

In [13]:
query = base_prompt_template.format(query=queries[4])

modelInput = tokenizer(query, return_tensors="pt").to("cuda")

outputs = model.generate(**modelInput, max_new_tokens = 500, do_sample = True)
print(tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
    You are a financial analyst designed to answer questions about business and finance. Your job is to reply to questions about finance topics and provide advice.

    Instruction: What is the rule of 72?
    
    Output:
<s> The rule of 72 is a financial calculation used to estimate how long it would take for an investment to double in value. It is based on the property that the amount of time it takes for an initial investment to double in value can be calculated by dividing 72 by the annual rate of return. For example, if the annual rate of return on an investment is 8%, then it will take 9 years for the investment to double in value (72/8). The rule of 72 is useful for understanding the potential compound growth of an investment, and can be used to compare different rates of return. It is also used to accurately predict the number of compounding periods it will take for an initial investment to double, triple, quadruple, etc. The rule of 72 is not perfect, but it is an easy w

In [14]:
query = base_prompt_template.format(query=queries[5])

modelInput = tokenizer(query, return_tensors="pt").to("cuda")

outputs = model.generate(**modelInput, max_new_tokens = 500, do_sample = True)
print(tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
    You are a financial analyst designed to answer questions about business and finance. Your job is to reply to questions about finance topics and provide advice.

    Instruction: What is CPI and how does it tell us what the inflation rate is?
    
    Output:
<s> Inflation doesn't have a single universal measure, but the official measure as used by the Federal Reserve Bank is called the CPI (Consumer Price Index). You can get the current measure here: http://www.bls.gov/cgi-bin/cpicalc.pl An example output would look something like this: If you've got a lot more or a lot less in monthly earnings now than a year ago, your "experienced inflation" this year will likely vary from these numbers, but these are what the Feds would consider "inflation" to be this year. They also publish "core inflation", which excludes food and energy costs, as these two are a bit more volatile. So you can go to the bls link and play with your own numbers and see why your experience may vary from the C

In [15]:
query = base_prompt_template.format(query=summary1)

modelInput = tokenizer(query, return_tensors="pt").to("cuda")

outputs = model.generate(**modelInput, max_new_tokens = 500, do_sample = True)
print(tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
    You are a financial analyst designed to answer questions about business and finance. Your job is to reply to questions about finance topics and provide advice.

    Instruction: "As of March 31, 2024, the Company had aggregate outstanding notes of $14,016 million, net of $61 million of issuance costs, with varying maturities (the "Notes"). Of the outstanding balance, $799 million, net of issuance costs, is classified as short-term debt on the Consolidated Balance Sheets. As of December 31, 2023, the Company had aggregate outstanding notes of $14,543 million, net of $65 million of issuance costs. Each of the Notes were issued at par and are senior unsecured obligations of the Company. Interest is payable semi-annually at fixed rates. A portion of the outstanding Notes is denominated in foreign currency (comprised of €5,170 million) and is remeasured into U.S. dollars at each balance sheet date (with remeasurement gain totaling $131 million for the three months ended March 31, 2

In [16]:
query = base_prompt_template.format(query=summary2)

modelInput = tokenizer(query, return_tensors="pt").to("cuda")

outputs = model.generate(**modelInput, max_new_tokens = 500, do_sample = True)
print(tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
    You are a financial analyst designed to answer questions about business and finance. Your job is to reply to questions about finance topics and provide advice.

    Instruction: "We anticipate our full-year 2024 capital expenditures will be in the range of $30-37 billion, a $2 billion increase of the high end of our prior range. We expect growth will be driven by investments in servers, including both AI and non-AI hardware, and data centers as we ramp up construction on sites with our previously announced new data center architecture. Our updated outlook reflects our evolving understanding of our artificial intelligence (AI) capacity demands as we anticipate what we may need for the next generations of foundational research and product development. While we are not providing guidance for years beyond 2024, we expect our ambitious long-term AI research and product development efforts will require growing infrastructure investments beyond this year." Based on this text what are

In [17]:
query = base_prompt_template.format(query=summary3)

modelInput = tokenizer(query, return_tensors="pt").to("cuda")

outputs = model.generate(**modelInput, max_new_tokens = 500, do_sample = True)
print(tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
    You are a financial analyst designed to answer questions about business and finance. Your job is to reply to questions about finance topics and provide advice.

    Instruction: "Product Liability We recorded liabilities of $596 million and $561 million in Accrued liabilities and Other liabilities at September 30, 2023 and December 31, 2022 for the expected cost of all known product liability claims, plus an estimate of the expected cost for product liability claims that have already been incurred and are expected to be filed in the future for which we are self-insured. It is reasonably possible that our accruals for product liability claims may increase in future periods in material amounts, although we cannot estimate a reasonable range of incremental loss based on currently available information. We believe that any judgment against us involving our products for actual damages will be adequately covered by our recorded accruals and, where applicable, excess liability insura